In [1]:
import sys
sys.path.append('..')
from osp import *

In [2]:
df_meta = get_corpus_metadata(periodize_by=50)
df_meta.period_discipline.value_counts()

period_discipline
1950-2000 Philosophy    15129
1950-2000 Literature    11888
2000-2050 Philosophy    11100
1900-1950 Literature     7538
1900-1950 Philosophy     6048
2000-2050 Literature     5917
Name: count, dtype: int64

In [39]:
words = get_non_content_words()
# words = ['hence','therefore','and','but','so']
data = get_counts_wordset(words)
data.head()

,the,of,more,an,than,but,for,him,as,his,...,somewheres,skywards,meself,yuk,northwestward,yippee,northeastward,southeastward,some-where,your-self
id,,,,,,,,,,,,,,,,,,,,,
phil/10.2307/2375809,0.165778,0.109275,0.004264,0.010661,0.006397,0.018124,0.022388,0.011194,0.017058,0.031983,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
phil/10.2307/45238188,0.103907,0.085619,0.004988,0.013023,0.007204,0.012746,0.022721,0.000554,0.016902,0.004156,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
phil/10.2307/20115538,0.142105,0.105263,0.006767,0.012782,0.000752,0.009774,0.023308,0.000000,0.014286,0.003008,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
phil/10.2307/40380502,0.118147,0.094773,0.011900,0.013175,0.009350,0.010625,0.026774,0.000000,0.028899,0.000425,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
phil/10.2307/24659549,0.155544,0.123432,0.001505,0.016056,0.006021,0.009032,0.014551,0.001505,0.021074,0.002007,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [40]:
def iter_binary_samples(data, colname='period',data_meta=None,balance=True, sample_size=None):
    data_meta = get_corpus_metadata() if data_meta is None else data_meta
    min_grp_size = min(data_meta.groupby(colname).size())
    data_meta = data_meta.groupby(colname).sample(n=min_grp_size)
    data_smpl = data.sample(frac=1)
    
    
    for coltype in data_meta[colname].unique():
        df_meta_yes = data_meta.query(f'{colname}==@coltype')
        df_meta_no = data_meta.query(f'{colname}!=@coltype')


        data_smpl_yes = data_smpl.loc[[i for i in df_meta_yes.index if i in data.index]].assign(_target=coltype)
        data_smpl_no = data_smpl.loc[[i for i in df_meta_no.index if i in data.index]].assign(_target=f'~{coltype}')

        if balance:
            if sample_size is None:
                minsize = min(data_smpl_yes.shape[0],data_smpl_no.shape[0])
                data_smpl_yes = data_smpl_yes.sample(n=minsize)
                data_smpl_no = data_smpl_no.sample(n=minsize)
            else:
                data_smpl_yes = data_smpl_yes.sample(n=sample_size, replace=True)
                data_smpl_no = data_smpl_no.sample(n=sample_size, replace=True)
        yield coltype, pd.concat([data_smpl_yes,data_smpl_no]).sample(frac=1)


In [41]:
by='period_discipline'
g,gfx = next(iter_binary_samples(data,by, sample_size=100))
print(g)
gfx._target.value_counts()
gfx

1900-1950 Literature


,the,of,more,an,than,but,for,him,as,his,...,skywards,meself,yuk,northwestward,yippee,northeastward,southeastward,some-where,your-self,_target
id,,,,,,,,,,,,,,,,,,,,,
lit/433459,0.236111,0.077381,0.001984,0.007937,0.001984,0.015873,0.015873,0.000000,0.019841,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1900-1950 Literature
lit/2872146,0.159677,0.095699,0.005376,0.003763,0.003763,0.014516,0.016667,0.008602,0.026344,0.032796,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,~1900-1950 Literature
phil/10.2307/20620183,0.157708,0.089738,0.003554,0.008441,0.003998,0.009773,0.019103,0.001333,0.021768,0.007552,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,~1900-1950 Literature
phil/10.2307/40040788,0.183333,0.060417,0.002083,0.016667,0.000000,0.006250,0.020833,0.000000,0.002083,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,~1900-1950 Literature
lit/458727,0.153032,0.083734,0.008662,0.013474,0.006256,0.008181,0.014437,0.007700,0.025505,0.034167,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1900-1950 Literature
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
lit/23356193,0.150578,0.100660,0.005776,0.009488,0.006188,0.013201,0.011139,0.006188,0.019802,0.016914,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,~1900-1950 Literature
lit/2871887,0.140990,0.124183,0.003735,0.008403,0.003735,0.014006,0.024276,0.005602,0.028011,0.034547,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,~1900-1950 Literature
lit/462100,0.135734,0.083102,0.008310,0.022161,0.002770,0.011080,0.019391,0.001385,0.033241,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,~1900-1950 Literature


In [42]:
def classify_binary_samples(data, colname='period',data_meta=None,balance=True, sample_size=None):
    iterr = iter_binary_samples(data, colname=colname,data_meta=data_meta,balance=balance, sample_size=sample_size)

    out_preds = []
    out_feats = []
    for g,gfx in tqdm(list(iterr),desc='classifying'):
        meta = {'test_group': g, 'test_group_type': colname}
        df_preds, df_feats = classify_data(gfx)
        out_preds.append(df_preds.assign(**meta))
        out_feats.append(df_feats.assign(**meta))
        # break
    return pd.concat(out_preds), pd.concat(out_feats)


In [45]:
df_preds, df_feats = classify_binary_samples(data,by, sample_size=5000)

classifying: 100%|██████████| 6/6 [05:23<00:00, 53.83s/it]


In [44]:
df_preds.groupby(['test_label']).mean(numeric_only=True).sort_values('accuracy',ascending=False)

,confidence,correct,accuracy
test_label,,,
2000-2050 Literature / ~2000-2050 Literature,0.871276,0.8670,0.8670
2000-2050 Philosophy / ~2000-2050 Philosophy,0.884573,0.8665,0.8665
1900-1950 Philosophy / ~1900-1950 Philosophy,0.868938,0.8580,0.8580
1900-1950 Literature / ~1900-1950 Literature,0.873848,0.8575,0.8575
1950-2000 Philosophy / ~1950-2000 Philosophy,0.801430,0.7795,0.7795
1950-2000 Literature / ~1950-2000 Literature,0.773151,0.7405,0.7405
